# Imports

In [295]:
import pandas as pd

In [296]:
import xml.etree.ElementTree as et

In [297]:
import matplotlib.pyplot as plt

In [298]:
import os

In [299]:
import math

# Functions

In [300]:
def transform_xml(xml_doc):
    attr = xml_doc.attrib
    for xml in xml_doc.iter('vehicle'):
        _dict = attr.copy()
        _dict.update(xml.attrib)
        
        yield _dict


In [301]:
price_per_liter = 624
eta = 0.45
m = 18000
g = 9.81
to_liter = 0.0000011765

In [302]:
def get_price_up(emission, delta_h):
    emission = emission + (1/eta) * m * g * delta_h
    emission = emission * price_per_liter
    print(emission)
    return emission * to_liter 

In [303]:
def get_price_down(emission, delta_h):
    emission = emission - (1/eta) * m * g * delta_h
    emission = emission * price_per_liter
    print(emission)
    return emission * to_liter

In [304]:
def get_last_folder(path):
    # Normalize the path to handle different separators and remove trailing separator
    normalized_path = os.path.normpath(path)
    # Split the path into components
    folders = normalized_path.split(os.sep)
    # Get the last folder
    last_folder = folders[-1]
    return last_folder

In [305]:
def calc_elevation_up(group):
    z_diff = pd.to_numeric(group['z']).diff()

    # Filter out negative differences (upward movement)
    up = z_diff.apply(lambda x: x if x > 0 else 0)

    # Sum the positive differences to get the total upward movement
    total_up = up.sum()
    return total_up

In [306]:
def calc_elevation_down(group):
    z_diff = pd.to_numeric(group['z']).diff()

    # Filter out negative differences (upward movement)
    down = z_diff.apply(lambda x: x if x < 0 else 0)

    # Sum the positive differences to get the total upward movement
    total_down = down.sum()
    return total_down

In [307]:
def transform_xml_tripinfo(xml_doc):
    attr = xml_doc.attrib
    for xml in xml_doc.iter('tripinfo'):
        _dict = attr.copy()
        _dict.update(xml.attrib)
        
        yield _dict

In [308]:
def get_routeids_df(base_folder):
    file_path = os.path.join(base_folder, "trips.txt")
    return pd.read_csv(file_path, sep=",")

In [309]:
def get_route_id(trip_id, trip_ids):
    trip_id = trip_id[:-2]
    line = trip_ids.loc[trip_ids['trip_id'] == trip_id]
    route_id = line.iloc[0]['route_id']

    return route_id

In [310]:
def transform_xml_stops(xml_doc):
    for route in xml_doc.iter('route'):
        route_dict = route.attrib.copy()
        stops = []
        
        # Iterate over each <stop> element within the current <route> element
        for stop in route.findall('stop'):
            stop_dict = stop.attrib.copy()
            stops.append(stop_dict)
        
        # Include stops in the route dictionary
        route_dict['stops'] = stops
        
        yield route_dict

# XML to df

In [311]:
base_folder = "C:\\Users\\Admin\\Sumo\\nap_gellert_e"

In [312]:
file_path = os.path.join(base_folder, "emission.out.xml")
emission_output = et.parse(file_path)

transform = transform_xml(emission_output.getroot())
emission_output_list = list(transform)

emission_output_df = pd.DataFrame(emission_output_list)
emission_output_df = emission_output_df.drop(emission_output_df.columns[0], axis=1)

emission_output_df.shape

(46317, 20)

In [313]:
emission_output_df

,id,eclass,CO2,CO,HC,NOx,PMx,fuel,electricity,noise,route,type,waiting,lane,pos,speed,angle,x,y,z
0,C767841140.0,HBEFA4/UBus_Std_gt15-18t_Euro-VI_A-C,2775.00,2.08,0.13,4.33,1.49,891.80,0.00,67.11,C767841140,MyBus,0.00,296910360#1_0,12.10,0.00,242.20,4054.57,1655.19,112.47
1,C767841140.0,HBEFA4/UBus_Std_gt15-18t_Euro-VI_A-C,8739.13,4.18,0.20,4.36,1.49,2808.43,0.00,74.03,C767841140,MyBus,0.00,296910360#1_0,13.25,1.15,242.20,4053.55,1654.65,112.53
2,C767841140.0,HBEFA4/UBus_Std_gt15-18t_Euro-VI_A-C,10793.46,4.02,0.21,4.43,1.50,3468.65,0.00,73.94,C767841140,MyBus,0.00,296910360#1_0,15.46,2.21,242.20,4051.60,1653.62,112.65
3,C767841140.0,HBEFA4/UBus_Std_gt15-18t_Euro-VI_A-C,12824.29,3.96,0.22,4.53,1.51,4121.32,0.00,74.15,C767841140,MyBus,0.00,296910360#1_0,18.68,3.22,242.20,4048.76,1652.12,112.82
4,C767841140.0,HBEFA4/UBus_Std_gt15-18t_Euro-VI_A-C,12844.47,3.61,0.22,4.59,1.52,4127.81,0.00,73.45,C767841140,MyBus,0.00,296910360#1_0,22.71,4.03,242.20,4045.20,1650.25,113.03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46312,C78105432.0,HBEFA4/UBus_Std_gt15-18t_Euro-VI_A-C,19739.29,3.88,0.28,5.26,1.62,6343.65,0.00,75.44,C7810510,MyBus,0.00,25440988#0_0,46.11,6.96,238.99,3710.75,3230.10,116.10
46313,C78105432.0,HBEFA4/UBus_Std_gt15-18t_Euro-VI_A-C,25839.52,4.42,0.34,5.88,1.71,8304.14,0.00,77.32,C7810510,MyBus,0.00,25440988#0_0,54.20,8.09,238.73,3703.88,3225.92,117.02
46314,C78105432.0,HBEFA4/UBus_Std_gt15-18t_Euro-VI_A-C,7438.26,2.10,0.19,4.64,1.57,2390.33,0.00,71.52,C7810510,MyBus,0.00,25440988#0_0,62.25,8.05,238.73,3697.04,3221.77,117.93
46315,C78105432.0,HBEFA4/UBus_Std_gt15-18t_Euro-VI_A-C,10491.91,2.48,0.22,4.86,1.60,3371.71,0.00,72.47,C7810510,MyBus,0.00,25440988#0_0,70.45,8.20,238.73,3690.08,3217.54,118.86


In [314]:
file_path = os.path.join(base_folder, "tripinfo.xml")
b_tripinfo_output = et.parse(file_path)

transform = transform_xml_tripinfo(b_tripinfo_output.getroot())
b_tripinfo_output_list = list(transform)

b_tripinfo_output_pd = pd.DataFrame(b_tripinfo_output_list)
b_tripinfo_output_pd = b_tripinfo_output_pd.drop(b_tripinfo_output_pd.columns[0], axis=1)
#b_tripinfo_output_pd

In [315]:
file_path = os.path.join(base_folder, "gtfs_pt_vehicles.add.xml")
stops = et.parse(file_path)

transform = transform_xml_stops(stops.getroot())
stops_list = list(transform)

stops_pd = pd.DataFrame(stops_list)

In [316]:
file_path = os.path.join(base_folder, "gtfs_pt_vehicles.add.xml")
vehicles = et.parse(file_path)

transform = transform_xml(vehicles.getroot())
vehicles_list = list(transform)

vehicles_pd = pd.DataFrame(vehicles_list)
vehicles_pd = vehicles_pd.drop(vehicles_pd.columns[0], axis=1)
#vehicles_pd

In [317]:
route_ids_df = get_routeids_df(base_folder)

# Group by id

In [318]:
grouped_df = emission_output_df.groupby('id')

list_of_dfs = [group_data for _, group_data in grouped_df]
#C74509106 = list_of_dfs[0]
#C74509124 = list_of_dfs[1]
#C74509135 = list_of_dfs[2]

In [319]:

results = []
for group_id, group_data in grouped_df:
    avg_speed = group_data['speed'].astype(float).mean()

    time_loss = b_tripinfo_output_pd.loc[b_tripinfo_output_pd['id'] == group_id, 'timeLoss'].values[0]
    route_length = b_tripinfo_output_pd.loc[b_tripinfo_output_pd['id'] == group_id, 'routeLength'].values[0]
    
    route = vehicles_pd.loc[vehicles_pd['id'] == group_id, 'route'].values[0]
    count_stops = stops_pd[stops_pd['id'] == route]['stops'].apply(len).sum()
    
    z_up = calc_elevation_up(group_data)
    z_down = calc_elevation_down(group_data)
    
    d_height = z_up + z_down
    
    fuel_sum = group_data['fuel'].astype(float).sum()
    
    route_id = get_route_id(group_id, route_ids_df)
    
    if(d_height >= 0):
        fuel_sum = get_price_up(group_data['fuel'].astype(float).sum(), d_height)
    else:
        fuel_sum = get_price_down(group_data['fuel'].astype(float).sum(), d_height)
    
    # Store the results in a dictionary
    group_result = {
        'routeid': route_id,
        'id': group_id,
        'avgSpeed': avg_speed,
        'fuel': fuel_sum,
        'timeloss': time_loss,
        'routeLength': route_length,
        'numOfStops': count_stops,
        'up': z_up,
        'down': z_down
    }
    
    # Append the dictionary to the results list
    results.append(group_result)

# Convert the results list to a DataFrame
result_df = pd.DataFrame(results)
print(result_df)


2397982884.9600005
2367533594.4000006
1160785416.9600005
812558504.1599976
903206285.2799975
8598043459.199999
255082694.87999833
10157298708.96
10138064526.72
10138119064.32
15623784246.720001
15665382832.319998
15699507039.839998
1956318143.5200005
3267705498.72
2075106939.3600016
3250659247.679998
2959523867.039999
1108690352.6400025
1090917391.2000034
2289600874.080001
3889003765.920001
2323579146.7200007
3184916473.4400015
3047517139.6800013
2393373715.200003
2305076860.320001
2290286781.1200004
2294709880.3200006
974108123.0399997
971093242.0799997
2467212059.520003
2471729744.639998
971371764.4799998
2430262785.599999
5600371345.919999
21298037854.08
50064342194.880005
32586294037.440006
51308985911.03999
51187285325.28001
21682095876.480003
1209334214.399999
7653131316.480002
7740651852.960002
8773235939.520002
1374026496.959999
1327435561.919999
1410966966.2399993
4401711163.200002
4228442318.8799996
2105493680.1600018
3348831988.32
3373755290.8799996
2009198724.000001
1473711

In [320]:
#avg_seed_b = C74509135['speed'].astype(float).mean()
#avg_seed_b

In [321]:
#float(C74509106['totalEnergyConsumed'].iloc[-1])-float(C74509106['totalEnergyRegenerated'].iloc[-1])
#24 2214.5173999999997  2219.5933
#06 2218.0583           2222.9716
#35 -615.606            -624.5073000000002

In [322]:
#plt.plot(C74509135['energyConsumed'].astype(float).cumsum())
#plt.xlabel('Time')
#plt.ylabel('Total Energy Regenerated')
#plt.title('Total Energy Regenerated Over Time')
#plt.xticks(rotation=45)
#plt.tight_layout()
#plt.show()

# CSV

In [323]:
tableEmission = pd.read_csv('emissionData.csv', delimiter=';')

tableEmission.shape

(0, 11)

## settings

In [324]:
locSetting = get_last_folder(base_folder)
seedSetting = 'dd'
trafficScaleSetting = 'wh'

#routeLengthSetting = :)
#numOfStopsSetting = :)
#tripIdSetting = :)
#avgSpeedSetting = :)
#timelossSetting = :)
#eleupSetting = 
#eledownSetting = 
#emissionSetting = :)

In [325]:
for index, row in result_df.iterrows():
    row_data = {
        'routeId': row['routeid'],
        'loc': locSetting,
        'tripId': row['id'],
        'seed': seedSetting,
        'avgSpeed': row['avgSpeed'],
        'timeloss': row['timeloss'],
        'route_length': row['routeLength'],
        'elevation_up': row['up'],
        'elevation_down': row['down'],
        'trafficScale': trafficScaleSetting,
        'numOfStops': row['numOfStops'],
        'emission': row['fuel']
    }
    temp_df = pd.DataFrame([row_data])
    #any empty or all-NA columns in tableEmission are excluded before concatenating the DataFrames
    tableEmission = tableEmission.dropna(axis=1, how='all')

    tableEmission = pd.concat([tableEmission, temp_df], ignore_index=True)


In [326]:
tableEmission.to_csv('batteryData.csv', index=False, sep=';')
tableEmission

,routeId,loc,tripId,seed,avgSpeed,timeloss,route_length,elevation_up,elevation_down,trafficScale,numOfStops,emission
0,0330,nap_gellert_e,C622152254.0,dd,4.476667,107.76,1023.22,18.70,-10.67,wh,3,2821.226864
1,0330,nap_gellert_e,C622155141.0,dd,4.791268,92.44,1023.22,18.62,-10.59,wh,3,2785.403274
2,9722,nap_gellert_e,C6305325.0,dd,4.905053,196.64,1866.71,17.58,-19.50,wh,4,1365.664043
3,2400,nap_gellert_e,C63598134.0,dd,4.487733,72.81,673.45,8.64,-10.76,wh,2,955.975080
4,2400,nap_gellert_e,C6359838.0,dd,4.182375,83.03,673.45,8.24,-10.86,wh,2,1062.622195
...,...,...,...,...,...,...,...,...,...,...,...,...
56,0085,nap_gellert_e,C78245617.0,dd,3.938365,774.54,8120.62,172.93,-219.05,wh,19,16604.427532
57,0085,nap_gellert_e,C78245945.0,dd,1.783998,3821.47,8685.16,239.14,-193.10,wh,18,18508.195696
58,1335,nap_gellert_e,C78257148.0,dd,5.509126,191.62,3407.49,99.13,-101.76,wh,4,2010.938496
59,1335,nap_gellert_e,C782571889.0,dd,5.169886,194.53,2724.58,87.49,-84.08,wh,4,1985.796812
